# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097009


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:12,  2.58s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:53,  2.05s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:36,  1.47s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:24,  1.05s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:00,  7.18it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  7.82it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00, 10.03it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  9.13it/s]

deivad                                0.052298
awright5                              0.000974
not-logged-in-d78ab5a3057206e8280b    1.883753
TSM_Ginga                             0.000287
rmz4160                               0.000287
Kartoon1x                             0.000272
not-logged-in-6cf01ec1c1cb535db647    0.003581
Slayz                                 0.074179
not-logged-in-bc5fdb9ea8d611e60b8b    0.000127
shirogane                             0.000309
not-logged-in-5a8bb7be433678ade709    0.000241
not-logged-in-9454a205d149d7907cd0    0.425943
not-logged-in-44c2a2616c2e5fbe7859    0.001254
not-logged-in-76a9c02fe5d62b1c2cae    0.000681
ComeMyChild                           0.001386
AlexYoung35                           0.000206
pangeli5                              0.000221
not-logged-in-3b0bff3092365bd5e47f    0.004428
acapirala                             0.000247
Susymuller                            0.008853
Lavadude                              0.032056
DrFloyd      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())